# Метрики для оценки алгоритмов машинного обучения в Python

### Постановка задачи
Рассмотреть различные метрики качества классификации, входящих в состав библиотеки scikit-learn. <br>
Для двух любых методов классификации из предыдущих работ и своего набора данных посчитать следующие метрики качества:<br>
    a. Точность классификации (Classification Accuracy)<br>
    b. Логарифм функции правдоподобия (Logarithmic Loss)<br>
    c. Область под кривой ошибок (Area Under ROC Curve)<br>
    d. Матрица неточностей (Confusion Matrix)<br>
    e. Отчет классификации (Classification Report)<br>
Для более точных результатов использовать кросс-валидацию; cравнить применимость используемых классификаторов, основываясь на полученных метриках
### Исходные данные
Датасет: https://archive.ics.uci.edu/ml/datasets/Website+Phishing<br>
Предметная область: Фишинговые сайты<br>
Задача: определить, фишинговый, подозрительный или нормальный сайт<br>
Количество записей: 1353<br>
Количество атрибутов: 9<br>
Атрибуты:
1. SFH {1,-1,0}
2. Pop-up Window  {1,-1,0}
3. SSL final state  {1,-1,0}
4. Request URL  {1,-1,0}
5. URL of Anchor  {1,-1,0}
6. Web traffic  {1,-1,0}
7. URL Length  {1,-1,0}
8. Age of domain  {1,-1}
9. Having IP Address {1,-1}

Во всех характеристиках значение «-1» означает «фишинговый», «0» - подозрительный, «1» - нормальный.<br>
### Описание параметров
__SFH (Server from handler)__ — Представление пользовательской информации, которая передается из веб страницы на сервер. Если оно пустое — сайт фишинговый, если передача идет на другой домен — подозрительный.
__Pop-up Window__ — Наличие всплывающего окна. Если при окне не доступен правый клик, то сайт фишинговый.<br>
__SSL final state__ — Подлинность SSL сертификата.<br>
__Request URL__ — Количество запросов к веб странице. Если их много, то, вероятно, сайт подвергся атаке, которая заменяет содержимое (текст/картинки). Если количество запросов велико — сайт фишинговый.<br>
__URL of Anchor__ — привязка к URL. Если при вводе адреса сайта в браузере происходит редирект на другой домен, то привязки нет. И если процент редиректов большой — сайт фишинговый.<br>
__Web traffic__ — объем веб трафиика сайта. У нормальных сайтов объем высокий, у фишинговых — низкий.<br>
__URL Length__ — Длина адреса сайта. Чем больше длина, тем выше вероятность, что в адрес встроен вредоносный код.<br>
__Age of domain__ — Возраст сайта. Если сайт существует менее полугода, то его можно заподозрить как фишинговый.<br>
__Having IP Address__ — Наличие IP адреса. Если адреса нет — сайт фишинговый.<br>

### Загрузка датасета

In [1]:
# coding=utf-8
from __future__ import division

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import preprocessing
from sklearn.svm import SVC


def load_data(filename):
    return pd.read_csv(filename, header=None).values


# разделение датасета на тестовую и обучающую выборку
def process_dataset(name):
    dataset = load_data(name)
    site_attr = dataset[:, :-1]  # список атрибутов для каждого сайта
    site_class = dataset[:, -1]  # класс (результат) сайта (норм, подозрительный, фишинговый)
    site_class = site_class.astype(np.float64, copy=False)
    site_attr = site_attr.astype(np.float64, copy=False)

    return site_attr, site_class


def train_split_dataset(site_attr, site_class, test_size, rnd_state):
    data_train, data_test, class_train, class_test = \
        train_test_split(site_attr, site_class, test_size=test_size, random_state=rnd_state)

    print_dataset_info(class_train, data_train)
    print_dataset_info(class_test, data_test)

    return data_train, data_test, class_train, class_test


def init(name):
    site_attr, site_class = process_dataset(name)
    print_dataset_info(site_class, site_attr)

    return site_attr, site_class

### Вывод информации о датасете

In [3]:
def print_dataset_info(site_class, site_attr):
    print('Number of records:', site_class.shape[0])
    print('Number of characters:', site_attr.shape[1])

    print('Class 0 (Normal): {:.2%}'.format(list(site_class).count(-1) / site_class.shape[0]))
    print('Class 1 (Suspicious): {:.2%}'.format(list(site_class).count(0) / site_class.shape[0]))
    print('Class 2 (Phishing): {:.2%}'.format(list(site_class).count(1) / site_class.shape[0]))


#### Whole dataset info
('Number of records:', 1353)<br>
('Number of characters:', 9)<br>
Class 0 (Normal): 51.88%<br>
Class 1 (Suspicious): 7.61%<br>
Class 2 (Phishing): 40.50%<br>
#### Training set
('Number of records:', 906)<br>
('Number of characters:', 9)<br>
Class 0 (Normal): 51.32%<br>
Class 1 (Suspicious): 8.17%<br>
Class 2 (Phishing): 40.51%<br>
#### Testing set
('Number of records:', 447)<br>
('Number of characters:', 9)<br>
Class 0 (Normal): 53.02%<br>
Class 1 (Suspicious): 6.49%<br>
Class 2 (Phishing): 40.49%<br>

### Получение метрик

#### Используемые методы классификации: _KNeighborsClassifier_, _LDA_

In [2]:
def get_metrics(site_attr, site_class, model, k_fold, scoring):
    result = cross_val_score(model, site_attr, site_class, cv=k_fold, scoring=scoring)
    print(" LDA:")
    print (" - mean: %0.5f" % result.mean())
    print (" - standart deviation: %0.5f" % result.std())
    result = cross_val_score(model, site_attr, site_class, cv=k_fold, scoring=scoring)
    print(" Neighbours:")
    print (" - mean: %0.5f" % result.mean())
    print (" - standart deviation: %0.5f" % result.std())
 

In [3]:
def get_confusion_matrix(data_train, data_test, class_train, class_test, model):
    model.fit(data_train, class_train)
    model_predicted = model.predict(data_test)
    model_matrix = confusion_matrix(class_test, model_predicted)
    print(model_matrix)
    return model_predicted


def get_classification_report(model_predicted, class_test):
    model_r = classification_report(class_test, model_predicted)
    print(model_r)

In [8]:
def main():
    site_attr, site_class = init('fs.dataset.csv')
    data_train, data_test, class_train, class_test = train_split_dataset(site_attr, site_class, 0.33, 77)
    k_fold = KFold(n_splits=10, random_state=77, shuffle=True)
    lda = LDA()
    neighbours = KNeighborsClassifier()
    
    print("Accuracy:")
    get_metrics(site_attr, site_class, lda, k_fold, 'accuracy')
    get_metrics(site_attr, site_class, neighbours, k_fold, 'accuracy')

    print("Logarithmic Loss:")
    get_metrics(site_attr, site_class, lda, k_fold, 'neg_log_loss')
    get_metrics(site_attr, site_class, neighbours, k_fold, 'neg_log_loss')

    print("Area Under ROC Curve:")
    get_metrics(site_attr, site_class, lda, k_fold, None)
    get_metrics(site_attr, site_class, neighbours, k_fold, None)

    print("Confusion Matrices:")
    print(" - LDA:")
    lda_predicted = get_confusion_matrix(data_train, data_test, class_train, class_test, lda)
    print(" - Neighbours:")
    neighbours_predicted = get_confusion_matrix(data_train, data_test, class_train, class_test, neighbours)

    print("Classification Reports:")
    print(" - LDA:")
    get_classification_report(lda_predicted, class_test)
    print(" - Neighbours:")
    get_classification_report(neighbours_predicted, class_test)

### Accuracy:

LDA:
 - mean: 0.82336
 - standart deviation: 0.02531 
 
Neighbours:
 - mean: 0.87363
 - standart deviation: 0.01809

### Logarithmic Loss:

LDA:
 - mean: -0.45486
 - standart deviation: 0.03024
    
Neighbours:
 - mean: -0.78234
 - standart deviation: 0.46922

### Area Under ROC Curve:

LDA:
 - mean: 0.82336
 - standart deviation: 0.02531
    
Neighbours:
 - mean: 0.87363
 - standart deviation: 0.01809

### Confusion Matrices
##### LDA:

\begin{vmatrix} 
204 &  1  & 18 \\
18 &  5  & 14 \\
21 &  2  & 164 \\
\end{vmatrix}

##### Neighbours:

\begin{vmatrix} 
204 &  4  & 15 \\
7 &  22  & 8 \\
15 &  5  & 167 \\
\end{vmatrix}

### Classification Reports:
##### LDA:

|              | precision      | recall  | f1-score | support |
| ------------ | -------------- | ------- | -------- | ------- |
| -1           | 0.84           |   0.91  |   0.88   |   223   |
| 0            | 0.62           |   0.14  |   0.22   |   37    |            
| 1            | 0.84           |   0.88  |   0.86   |   187   | 
|              |                |         |          |         |
|avg / total   |    0.82        |0.83     | 0.81     | 447

##### Neighbours:

|              | precision      | recall  | f1-score | support |
| ------------ | -------------- | ------- | -------- | ------- |
| -1           | 0.90           |   0.91  |   0.91   |   223   |
| 0            | 0.71           |   0.59  |   0.65   |   37    |            
| 1            | 0.88           |   0.89  |   0.89   |   187   | 
|              |                |         |          |         |
|avg / total   |    0.88        |0.88     | 0.88     | 447

#### Вывод
Согласно результатам, полученным в ходе лабораторной работы, можно сделать вывод, что на исследуемом датасете основные метрические показатели (точность логарифм функции правдоподобия, область под кривой ошибок) у метода ближайших соседей выше, чем у метода линейного дискриминантного анализа. По матрице неточностей был построен отчет классификации, иллюстрирующий показатели точности классификации и полноты. Из данного отчета также видно, что метод ближайших соседей на данном датасете оказался точнее - 0.88 против 0.82. \